In [58]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import warnings
import datetime
import geopandas as gpd
import folium
import time
import my_code
warnings.filterwarnings('ignore')
from scipy.stats.stats import pearsonr 
import seaborn as sns

data = pd.read_csv('/Users/alket/Desktop/dati/new_data2_Modena_backfill_forwfill.csv',index_col = 0)
data

,date,cell_num,nr_people,hours,minutes
0,2017-04-16,389-1015,361.0,0.0,0.0
1353,2017-04-16,389-1015,367.0,0.0,15.0
2706,2017-04-16,389-1015,359.0,0.0,30.0
4059,2017-04-16,389-1015,354.0,0.0,45.0
5412,2017-04-16,389-1015,350.0,1.0,0.0
...,...,...,...,...,...
15970811,2017-08-16,421-1055,17.0,22.0,45.0
15972164,2017-08-16,421-1055,17.0,23.0,0.0
15973517,2017-08-16,421-1055,17.0,23.0,15.0
15974870,2017-08-16,421-1055,18.0,23.0,30.0


In [67]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15976224 entries, 0 to 15976223
Data columns (total 5 columns):
date         object
cell_num     object
nr_people    float64
hours        float64
minutes      float64
dtypes: float64(3), object(2)
memory usage: 731.3+ MB


In [59]:
agg_by_cell = data.groupby(['cell_num'])
counter = 0
for i, k in agg_by_cell: 
    print(i, k)
    counter += 1
    if counter > 1 : break

389-1015                 date  cell_num  nr_people  hours  minutes
0         2017-04-16  389-1015      361.0    0.0      0.0
1353      2017-04-16  389-1015      367.0    0.0     15.0
2706      2017-04-16  389-1015      359.0    0.0     30.0
4059      2017-04-16  389-1015      354.0    0.0     45.0
5412      2017-04-16  389-1015      350.0    1.0      0.0
...              ...       ...        ...    ...      ...
15969459  2017-08-16  389-1015      269.0   22.0     45.0
15970812  2017-08-16  389-1015      268.0   23.0      0.0
15972165  2017-08-16  389-1015      271.0   23.0     15.0
15973518  2017-08-16  389-1015      266.0   23.0     30.0
15974871  2017-08-16  389-1015      266.0   23.0     45.0

[11808 rows x 5 columns]
389-1016                 date  cell_num  nr_people  hours  minutes
1         2017-04-16  389-1016      188.0    0.0      0.0
1354      2017-04-16  389-1016      191.0    0.0     15.0
2707      2017-04-16  389-1016      186.0    0.0     30.0
4060      2017-04-16  389-10

In [60]:
def get_corr_adiacent_cells(cell, index):
    counter = 0
    adiacent_cells = []
    s = cell.split('-')
    row = int(s[0])
    col = int(s[1])
    rowup = row+index
    rowdown = row-index
    coldes = col+index
    colsin = col-index
    cell1 = str(row)+'-'+str(coldes)
    cell2 = str(row)+'-'+str(colsin)
    cell3 = str(rowup)+'-'+str(col)
    cell4 = str(rowdown)+'-'+str(col)
    
    cell5 = str(rowup)+'-'+str(coldes)
    cell6 = str(rowup)+'-'+str(colsin)
    cell7 = str(rowdown)+'-'+str(coldes)
    cell8 = str(rowdown)+'-'+str(colsin)
    adiacent_cells.extend([cell1, cell2, cell3, cell4, cell5, cell6, cell7, cell8])
        
    return adiacent_cells

In [61]:
def crosscorr(datax, datay, lag=0):
    """ Lag-N cross correlation. 
    Parameters
    ----------
    lag : int, default 0
    datax, datay : pandas.Series objects of equal length

    Returns
    ----------
    crosscorr : float
    """
    return datax.corr(datay.shift(lag))

mylist = pd.Series([1, 2, 3, 4, 5, 6, 7])
mylist2 = pd.Series([1, 2, 3, 4, 5, 6, 7])
crosscorr(mylist, mylist2, lag = 2)
type(mylist2)

pandas.core.series.Series

In [79]:
cell1 = agg_by_cell.get_group('400-1035')
cell2 = agg_by_cell.get_group('400-1036')

cell3 = agg_by_cell.get_group('403-1036')
cell4 = agg_by_cell.get_group('406-1031')
cell5 = agg_by_cell.get_group('409-1041')

cell1_series = cell1['nr_people'].values
cell2_series = cell2['nr_people'].values
cell3_series = cell3['nr_people'].values
cell4_series = cell4['nr_people'].values
cell5_series = cell5['nr_people'].values

ls = [cell1_series, cell2_series, cell3_series, cell4_series, cell5_series]
corr = np.corrcoef(ls)
corr[0]

array([1.        , 0.99819136, 0.88760344, 0.69595579, 0.77194584])

In [16]:
cells2be_correlated = ['400-1035', '400-1036', '400-1037', '400-1028', '400-1029', '400-1032',
                       '401-1035', '401-1038', '401-1041', '401-1033', '401-1029', '401-1032',
                       '402-1043', '402-1039', '402-1033', '402-1035', '402-1037', '402-1031',
                       '403-1043', '403-1031', '403-1034', '403-1036', '403-1037', '403-1039',
                       '404-1036', '404-1037', '404-1038', '404-1040', '404-1042', '404-1035',
                       '409-1036', '409-1037', '409-1038', '409-1040', '409-1042', '409-1035',
                       '408-1036', '408-1037', '408-1038', '408-1040', '408-1042', '408-1035',
                       '407-1035', '407-1037', '407-1039', '407-1041', '407-1042', '407-1036',
                       '405-1031', '405-1032', '405-1034', '405-1036', '405-1038', '405-1042',
                       '406-1034', '406-1031', '406-1033', '406-1035', '406-1039', '406-1040']

In [42]:
data2dict = {}
for num in range(5):
    num = num+1
    corrlist = []
    for cell2c in cells2be_correlated:
        
        adiacent_cells = get_corr_adiacent_cells(cell2c, num)
        print(adiacent_cells)
        my_cell = agg_by_cell.get_group(cell2c)
        my_cell_ts = my_cell['nr_people']
        print(type(my_cell_ts))
        print(my_cell_ts[:10])
        for i in adiacent_cells: 
            cell_i = agg_by_cell.get_group(i)
            
            #print(cell_i[:10])
            pc = crosscorr(my_cell['nr_people'],cell_i['nr_people'] , num)
            print(pc)
            corrlist.append(pc)
            
    print(num , np.mean(np.array(corrlist)))
    data2dict[num] = np.mean(np.array(corrlist))

['400-1036', '400-1034', '401-1035', '399-1035', '401-1036', '401-1034', '399-1036', '399-1034']
<class 'pandas.core.series.Series'>
471      707.0
1824     704.0
3177     696.0
4530     679.0
5883     668.0
7236     631.0
8589     612.0
9942     601.0
11295    592.0
12648    571.0
Name: nr_people, dtype: float64
nan
nan
nan
nan
nan
nan
nan
nan
['400-1037', '400-1035', '401-1036', '399-1036', '401-1037', '401-1035', '399-1037', '399-1035']
<class 'pandas.core.series.Series'>
472      938.0
1825     934.0
3178     923.0
4531     901.0
5884     887.0
7237     837.0
8590     813.0
9943     797.0
11296    784.0
12649    756.0
Name: nr_people, dtype: float64
nan
nan
nan
nan
nan
nan
nan
nan
['400-1038', '400-1036', '401-1037', '399-1037', '401-1038', '401-1036', '399-1038', '399-1036']
<class 'pandas.core.series.Series'>
473      1120.0
1826     1119.0
3179     1093.0
4532     1095.0
5885     1086.0
7238     1072.0
8591     1064.0
9944     1052.0
11297    1043.0
12650    1044.0
Name: nr_peop

nan
nan
nan
nan
nan
nan
nan
['409-1039', '409-1037', '410-1038', '408-1038', '410-1039', '410-1037', '408-1039', '408-1037']
<class 'pandas.core.series.Series'>
843      402.0
2196     397.0
3549     397.0
4902     398.0
6255     397.0
7608     403.0
8961     401.0
10314    404.0
11667    408.0
13020    411.0
Name: nr_people, dtype: float64
nan
nan
nan
nan
nan
nan
nan
nan
['409-1041', '409-1039', '410-1040', '408-1040', '410-1041', '410-1039', '408-1041', '408-1039']
<class 'pandas.core.series.Series'>
845      105.0
2198     105.0
3551     105.0
4904     105.0
6257     105.0
7610     108.0
8963     108.0
10316    108.0
11669    111.0
13022    112.0
Name: nr_people, dtype: float64
nan
nan
nan
nan
nan
nan
nan
nan
['409-1043', '409-1041', '410-1042', '408-1042', '410-1043', '410-1041', '408-1043', '408-1041']
<class 'pandas.core.series.Series'>
847      174.0
2200     174.0
3553     174.0
4906     175.0
6259     176.0
7612     178.0
8965     178.0
10318    178.0
11671    181.0
13024    1

nan
nan
nan
nan
nan
nan
['400-1039', '400-1035', '402-1037', '398-1037', '402-1039', '402-1035', '398-1039', '398-1035']
<class 'pandas.core.series.Series'>
473      1120.0
1826     1119.0
3179     1093.0
4532     1095.0
5885     1086.0
7238     1072.0
8591     1064.0
9944     1052.0
11297    1043.0
12650    1044.0
Name: nr_people, dtype: float64
nan
nan
nan
nan
nan
nan
nan
nan
['400-1030', '400-1026', '402-1028', '398-1028', '402-1030', '402-1026', '398-1030', '398-1026']
<class 'pandas.core.series.Series'>
464      382.0
1817     386.0
3170     388.0
4523     387.0
5876     389.0
7229     388.0
8582     388.0
9935     389.0
11288    390.0
12641    391.0
Name: nr_people, dtype: float64
nan
nan
nan
nan
nan
nan
nan
nan
['400-1031', '400-1027', '402-1029', '398-1029', '402-1031', '402-1027', '398-1031', '398-1027']
<class 'pandas.core.series.Series'>
465      320.0
1818     324.0
3171     325.0
4524     324.0
5877     326.0
7230     325.0
8583     326.0
9936     326.0
11289    327.0
1264

nan
nan
nan
nan
nan
nan
nan
nan
['409-1044', '409-1040', '411-1042', '407-1042', '411-1044', '411-1040', '407-1044', '407-1040']
<class 'pandas.core.series.Series'>
847      174.0
2200     174.0
3553     174.0
4906     175.0
6259     176.0
7612     178.0
8965     178.0
10318    178.0
11671    181.0
13024    182.0
Name: nr_people, dtype: float64
nan
nan
nan
nan
nan
nan
nan
nan
['409-1037', '409-1033', '411-1035', '407-1035', '411-1037', '411-1033', '407-1037', '407-1033']
<class 'pandas.core.series.Series'>
840      497.0
2193     494.0
3546     498.0
4899     495.0
6252     498.0
7605     504.0
8958     510.0
10311    512.0
11664    513.0
13017    514.0
Name: nr_people, dtype: float64
nan
nan
nan
nan
nan
nan
nan
nan
['408-1038', '408-1034', '410-1036', '406-1036', '410-1038', '410-1034', '406-1038', '406-1034']
<class 'pandas.core.series.Series'>
800      540.0
2153     536.0
3506     541.0
4859     537.0
6212     541.0
7565     548.0
8918     553.0
10271    556.0
11624    557.0
12977 

nan
nan
nan
nan
nan
nan
nan
nan
['401-1038', '401-1032', '404-1035', '398-1035', '404-1038', '404-1032', '398-1038', '398-1032']
<class 'pandas.core.series.Series'>
512      549.0
1865     550.0
3218     542.0
4571     530.0
5924     521.0
7277     494.0
8630     478.0
9983     469.0
11336    466.0
12689    450.0
Name: nr_people, dtype: float64
nan
nan
nan
nan
nan
nan
nan
nan
['401-1041', '401-1035', '404-1038', '398-1038', '404-1041', '404-1035', '398-1041', '398-1035']
<class 'pandas.core.series.Series'>
515      933.0
1868     936.0
3221     918.0
4574     920.0
5927     907.0
7280     886.0
8633     873.0
9986     861.0
11339    857.0
12692    856.0
Name: nr_people, dtype: float64
nan
nan
nan
nan
nan
nan
nan
nan
['401-1044', '401-1038', '404-1041', '398-1041', '404-1044', '404-1038', '398-1044', '398-1038']
<class 'pandas.core.series.Series'>
518      521.0
1871     518.0
3224     513.0
4577     504.0
5930     501.0
7283     500.0
8636     497.0
9989     492.0
11342    488.0
12695 

nan
nan
nan
nan
nan
nan
nan
['408-1041', '408-1035', '411-1038', '405-1038', '411-1041', '411-1035', '405-1041', '405-1035']
<class 'pandas.core.series.Series'>
802      372.0
2155     369.0
3508     369.0
4861     373.0
6214     373.0
7567     383.0
8920     382.0
10273    383.0
11626    386.0
12979    392.0
Name: nr_people, dtype: float64
nan
nan
nan
nan
nan
nan
nan
nan
['408-1043', '408-1037', '411-1040', '405-1040', '411-1043', '411-1037', '405-1043', '405-1037']
<class 'pandas.core.series.Series'>
804      261.0
2157     263.0
3510     261.0
4863     263.0
6216     263.0
7569     272.0
8922     270.0
10275    271.0
11628    277.0
12981    282.0
Name: nr_people, dtype: float64
nan
nan
nan
nan
nan
nan
nan
nan
['408-1045', '408-1039', '411-1042', '405-1042', '411-1045', '411-1039', '405-1045', '405-1039']
<class 'pandas.core.series.Series'>
806      411.0
2159     410.0
3512     409.0
4865     411.0
6218     413.0
7571     418.0
8924     416.0
10277    417.0
11630    422.0
12983    4

nan
nan
nan
nan
nan
nan
nan
nan
['401-1033', '401-1025', '405-1029', '397-1029', '405-1033', '405-1025', '397-1033', '397-1025']
<class 'pandas.core.series.Series'>
506      544.0
1859     549.0
3212     548.0
4565     548.0
5918     548.0
7271     551.0
8624     553.0
9977     554.0
11330    557.0
12683    557.0
Name: nr_people, dtype: float64
nan
nan
nan
nan
nan
nan
nan
nan
['401-1036', '401-1028', '405-1032', '397-1032', '405-1036', '405-1028', '397-1036', '397-1028']
<class 'pandas.core.series.Series'>
509      518.0
1862     523.0
3215     526.0
4568     526.0
5921     520.0
7274     501.0
8627     494.0
9980     489.0
11333    484.0
12686    478.0
Name: nr_people, dtype: float64
nan
nan
nan
nan
nan
nan
nan
nan
['402-1047', '402-1039', '406-1043', '398-1043', '406-1047', '406-1039', '398-1047', '398-1039']
<class 'pandas.core.series.Series'>
561      656.0
1914     652.0
3267     645.0
4620     640.0
5973     645.0
7326     634.0
8679     632.0
10032    630.0
11385    632.0
12738 

nan
nan
nan
nan
nan
nan
nan
['407-1041', '407-1033', '411-1037', '403-1037', '411-1041', '411-1033', '403-1041', '403-1033']
<class 'pandas.core.series.Series'>
760      493.0
2113     493.0
3466     495.0
4819     497.0
6172     498.0
7525     497.0
8878     495.0
10231    493.0
11584    493.0
12937    493.0
Name: nr_people, dtype: float64
nan
nan
nan
nan
nan
nan
nan
nan
['407-1043', '407-1035', '411-1039', '403-1039', '411-1043', '411-1035', '403-1043', '403-1035']
<class 'pandas.core.series.Series'>
762      361.0
2115     360.0
3468     360.0
4821     365.0
6174     366.0
7527     377.0
8880     377.0
10233    378.0
11586    382.0
12939    388.0
Name: nr_people, dtype: float64
nan
nan
nan
nan
nan
nan
nan
nan
['407-1045', '407-1037', '411-1041', '403-1041', '411-1045', '411-1037', '403-1045', '403-1037']
<class 'pandas.core.series.Series'>
764      366.0
2117     367.0
3470     366.0
4823     368.0
6176     368.0
7529     374.0
8882     372.0
10235    374.0
11588    376.0
12941    3

nan
nan
nan
nan
nan
nan
['402-1040', '402-1030', '407-1035', '397-1035', '407-1040', '407-1030', '397-1040', '397-1030']
<class 'pandas.core.series.Series'>
553      643.0
1906     649.0
3259     640.0
4612     637.0
5965     625.0
7318     599.0
8671     583.0
10024    574.0
11377    569.0
12730    558.0
Name: nr_people, dtype: float64
nan
nan
nan
nan
nan
nan
nan
nan
['402-1042', '402-1032', '407-1037', '397-1037', '407-1042', '407-1032', '397-1042', '397-1032']
<class 'pandas.core.series.Series'>
555      696.0
1908     699.0
3261     687.0
4614     689.0
5967     676.0
7320     654.0
8673     640.0
10026    630.0
11379    625.0
12732    620.0
Name: nr_people, dtype: float64
nan
nan
nan
nan
nan
nan
nan
nan
['402-1036', '402-1026', '407-1031', '397-1031', '407-1036', '407-1026', '397-1036', '397-1026']
<class 'pandas.core.series.Series'>
549      634.0
1902     638.0
3255     638.0
4608     638.0
5961     640.0
7314     647.0
8667     651.0
10020    652.0
11373    653.0
12726    655.0

<class 'pandas.core.series.Series'>
759      444.0
2112     443.0
3465     447.0
4818     446.0
6171     450.0
7524     450.0
8877     456.0
10230    456.0
11583    455.0
12936    454.0
Name: nr_people, dtype: float64
nan
nan
nan
nan
nan
nan
nan
nan
['405-1036', '405-1026', '410-1031', '400-1031', '410-1036', '410-1026', '400-1036', '400-1026']
<class 'pandas.core.series.Series'>
672      629.0
2025     629.0
3378     629.0
4731     630.0
6084     632.0
7437     638.0
8790     645.0
10143    646.0
11496    646.0
12849    649.0
Name: nr_people, dtype: float64
nan
nan
nan
nan
nan
nan
nan
nan
['405-1037', '405-1027', '410-1032', '400-1032', '410-1037', '410-1027', '400-1037', '400-1027']
<class 'pandas.core.series.Series'>
673      666.0
2026     666.0
3379     668.0
4732     669.0
6085     671.0
7438     679.0
8791     688.0
10144    690.0
11497    689.0
12850    692.0
Name: nr_people, dtype: float64
nan
nan
nan
nan
nan
nan
nan
nan
['405-1039', '405-1029', '410-1034', '400-1034', '410-10